In [24]:
import scipy.io
import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
mat = scipy.io.loadmat('Data_Ankle.mat')

In [3]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue May 30 12:06:20 2023',
 '__version__': '1.0',
 '__globals__': [],
 'ankle_data': array([[ 0.00000000e+00, -1.03213808e+01, -9.42861914e-02, ...,
          8.25063296e-01, -1.49258824e+00, -2.01162801e+00],
        [ 1.00000000e-02, -7.99031164e+00, -9.61738741e-02, ...,
          8.22468551e-01, -1.49319212e+00, -3.65520937e+00],
        [ 2.00000000e-02, -6.20873764e+00, -9.64488565e-02, ...,
          8.10996502e-01, -1.49586655e+00, -5.78643292e+00],
        ...,
        [ 4.79970000e+02,  5.56958911e+02, -1.12156787e-02, ...,
          2.25437228e-01, -1.68091209e+00,  4.26107807e+01],
        [ 4.79980000e+02,  5.64941734e+02, -5.72538797e-03, ...,
          2.15807917e-01, -1.68371085e+00,  4.40519172e+01],
        [ 4.79990000e+02,  5.74057993e+02, -4.47411580e-03, ...,
          2.08124660e-01, -1.68575417e+00,  4.51366601e+01]])}

In [37]:
data = pd.DataFrame(mat['ankle_data'])
data.shape

(48000, 7)

In [47]:
class LeggedMotion_dataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        """
         7 different data, being time, 
         vertical ground reaction force in Newton, 
         trunk joint angle, hip joint angle,
         knee joint angle, ankle joint angle (all in radians),
         and the joint moment in Nm.
        """
        self.transform = transform
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):

        three_tuple = []
        # 
        features = torch.tensor(self.data.iloc[idx])
        #print("features:", features.shape)
        # 
        return (features[0], features[1], features[2], features[3], features[4], features[5], features[6])


In [48]:
train_data, test_data = train_test_split(data, test_size=0.333333, random_state=1)
train_batches = torch.utils.data.DataLoader(LeggedMotion_dataset(data=train_data), batch_size=50,  num_workers=4)

In [80]:
x = torch.randn(1).cuda()
x

tensor([-0.1668], device='cuda:0')

In [82]:
x.detach().item()

-0.16681794822216034

In [71]:
torch.hstack((torch.randn(5).unsqueeze(1), torch.randn(5).unsqueeze(1))).size()

torch.Size([5, 2])

In [51]:
for b in train_batches:
    time, ground_reac, trunk_j, hip_j,knee_j, ankle_j, joint_moment = b
    #labels = b[1]
    print(trunk_j.shape)

torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50])
torch.Size([50